In [6]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='../../dataset/mnist', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='../../dataset/mnist', 
                           train=False, 
                           transform=transforms.ToTensor())

# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [13]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return F.log_softmax(out) 
        # CrossEntropy 내부에서 softmax 계산해주는 모양인데 log_softmax 쓰면 로스가 더 줄어든다
        # 뭐지..? 그냥 softmax 쓰면 중첩되서 로스가 안줄어듬

In [14]:
model = LogisticRegression(input_size, num_classes)

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [15]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch: [1/5], Step: [100/600], Loss: 2.2386
Epoch: [1/5], Step: [200/600], Loss: 2.1150
Epoch: [1/5], Step: [300/600], Loss: 2.0211
Epoch: [1/5], Step: [400/600], Loss: 1.9450
Epoch: [1/5], Step: [500/600], Loss: 1.8529
Epoch: [1/5], Step: [600/600], Loss: 1.7382
Epoch: [2/5], Step: [100/600], Loss: 1.7467
Epoch: [2/5], Step: [200/600], Loss: 1.6677
Epoch: [2/5], Step: [300/600], Loss: 1.5661
Epoch: [2/5], Step: [400/600], Loss: 1.5083
Epoch: [2/5], Step: [500/600], Loss: 1.5519
Epoch: [2/5], Step: [600/600], Loss: 1.5156
Epoch: [3/5], Step: [100/600], Loss: 1.3911
Epoch: [3/5], Step: [200/600], Loss: 1.3999
Epoch: [3/5], Step: [300/600], Loss: 1.3163
Epoch: [3/5], Step: [400/600], Loss: 1.2420
Epoch: [3/5], Step: [500/600], Loss: 1.3027
Epoch: [3/5], Step: [600/600], Loss: 1.2045
Epoch: [4/5], Step: [100/600], Loss: 1.1302
Epoch: [4/5], Step: [200/600], Loss: 1.1967
Epoch: [4/5], Step: [300/600], Loss: 1.1152
Epoch: [4/5], Step: [400/600], Loss: 1.1326
Epoch: [4/5], Step: [500/600], L

In [18]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the model on the 10000 test images: 82 %
